# 01_Segmentation_and_the_Napari_image_viewer

***

In this Notebook, we will look at some $S. cerevisiae$ cells to which a ribosomal dye has been applied. As yeast cells are filled with 10.000s of ribosomal proteins this should lead to a strong increase in signal. However, a **negative control** image was also included in this dataset, which demonstrates the expected background signal in cells without any staining. We want to measure how fluorescence intensity between the stained cells and negative control cells differs. The main concepts that we would like you to learn from this notebook are what is segmentation, how to extract data from segmentation masks, and what the python viewer Napari and its plugins are capable of.

In [ ]:
from glob import glob
from skimage import io
from pathlib import Path
import napari
from napari.layers import Image, Labels
import numpy as np
from skimage.measure import regionprops_table as rt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import label
from magicgui import magicgui

## <font color='seagreen'> 0: Loading and inspecting Images </font>

In [ ]:
path=r'.\data\01_Segmentation_and_the_Napari_image_viewer\*.tif'
img_paths=glob(path)
img_paths

imgs=[io.imread(img_path) for img_path in img_paths]

In [ ]:
imgs[0].shape

In [ ]:
fig,axes=plt.subplots(ncols=imgs[0].shape[0],figsize=(20,20)) 

names= ['ribosomal stain', 'nuclear stain', 'DIC', 'cell mask', 'nuclear mask']

i=0
for ax,name in zip(axes,names):
    ax.imshow(imgs[0][i])
    ax.set_title(f'channel : {name}')
    i+=1

The last two channels are segmented masks based on the provided DIC and nuclear stain. A segmentation mask is an image in which every object of interest is given an unique label id, whilst the background is given the value zero. This allows us to count the number of object per image image, but also extract interesting variables about cell morphology or fluorescence intensity. At the end of the notebook we will show you how to produce the masks yourself. But first, we will see how we can use these masks to exract data from our images.

## <font color='seagreen'> 1: Extracting data from Masks </font>

For extraxting the data from our images we will use the funnction regionprops_table from the scikit-image processing library. Within this function we can select which properties are to be extracted. Below we extract the properties : **label**, **mean_intensity**, **eccentricity** and **centroid**. The exercise below will help you understand what these properties are.

In [ ]:
#index fluorescent channels
CY5,DAPI,DIC,cell_mask,nuclear_mask,=0,1,2,3,4

#loop through images and extract properties of each labelled cell
dfs=[]
for img, i in zip(imgs,range(len(imgs))):
    #create pandas dataframe extracting information from image labels and fluorescent channel. rt is the function regionprops_table
    df=pd.DataFrame(rt(img[cell_mask].astype(int), img[CY5], properties=['label','mean_intensity','eccentricity','centroid']))
    df['image_index']=i
    dfs.append(df)

#pd.concatenate all dataframes into one big frame
df=pd.concat(dfs).reset_index()

#summarize the contents of this big frame
df.describe()

In [ ]:
#show the first 6 entries of the dataframe
df.head(6)

**Q1.0: How many cells were imaged in total?**

type here

In [ ]:
sns.stripplot(data=df,x='image_index', y='mean_intensity',alpha=0.5, s=3, color='black')
sns.boxplot(data=df,x='image_index', y='mean_intensity',color='purple')

plt.title('CY5 Fluorescence intensity per image')

**Q1.1: Which image contains the negative control unstained cells?**

type here

**Q1.2: Can you plot all the negative control cells against ribosomal stained cells?**

In [ ]:
#code here

**Q1.3: Cell masks can also be used to study cell morphology. What kind of cell morphology can be described with eccentricity?**

type here

In [ ]:
sns.stripplot(data=df,x='image_index', y='eccentricity',color='black', alpha=0.5, s=3)
sns.boxplot(data=df,x='image_index', y='eccentricity')

plt.title('Eccenctricity distribution per image')

**Q0.3: Can you create a crop of a cell based on the information in the create dataframe? Fill in the code below**

In [ ]:
#displays the columns of entry number 20 in your dataframe
df.iloc[20]

In [ ]:
#fill in these variables. Use values from the table above
image_index=
centroid0=
centroid1=


plt.imshow(imgs[int(image_index)][0,int(centroid0)-50:int(centroid0)+50,int(centroid1)-50:int(centroid1)+50])
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('fluorescence (A.U)')

**Q1.4: What cell property is described by centroid?**

type here

## <font color='seagreen'> 2: the Napari image viewer</font> 

In the next section, we will introduce to you the  napari image viewer and its plugin hub. Napari is a python based image viewer with a wide community of develpopers focused on image analysis. Therefore, there is a huge amount of plugins avalaible for all kinds of different image analysis problems (https://www.napari-hub.org/). Here, we will show you how open image in napari and perform segmentation on our images using the napari-Cellpose plugin. Cellpose is Convolutional Neural Network (deep-learning) based segmentation algorithm to produce cell mask. Next, we will use your own code to create a plugin yourself to produce the a nuclear mask.

In [ ]:
#create an instance of the napari viewer and add the first image
viewer = napari.Viewer()

#img index 
img= imgs[0]

viewer.add_image(img[CY5], name='CY5') 
viewer.add_image(img[DAPI], name='DAPI') 
viewer.add_image(img[DIC], name='DIC') 

In [ ]:
#if napari doesn't open try to reintall the package.
!pip install "napari[all]"

In the top layer of the created viewer you can find the button **Plugins**. Click this button, and select the plugin named cellpose. Next, switch you attention to the created window on the right hand side of the viewer. Here, in the image-layer dropdown menu select the **CY5** channel. Then press the button **run segmentation**. Be patient as it can take a while produce its resutls. 

**Q2.0: Hover over the produced masks, do you understand what it means that each cell is assigned its own unique label?**

type here

Lets create our own plugin to perform nuclear mask segmentation. For this purpose we will first close the cellpose window within the viewer. Next, you can execute the code block below to add our own plugin to the viewer. This is done with a python library called magicgui. This library opens up exciting avenues for thos who want to quickly develop their own plugins. For an overview of what is possible with magicgui see: https://pyapp-kit.github.io/magicgui/.

In [ ]:
@magicgui(call_button="threshold", threshold={'widget_type':'Slider',"max": 100},auto_call=False)
def threshold(image: Image, threshold: int = 100) -> Labels:
    """ thresholding widget for nuclear segmentation"""

    image =(image.data*(100/image.data.max()))
    thresholded_image=image>threshold
    nuclei,_=label(thresholded_image)
    
    return Labels(nuclei, name='nuclear_mask')

viewer.window.add_dock_widget(threshold)

**Q2.1: Where you able to reproduce the nuclear masks of a single image? Can you think of a use case for nuclear mask in an image data analyis pipeline?**

type here

***

## <font color='seagreen'> 3: Programming excersize </font> 

**Q3.0: Extract the area of each nuclei in the nuclear_masks channel of our images. Then plot these areas as a histogram.** 

In [ ]:
code here

***

## <font color='seagreen'> 4: Final conceptual questions</font> 

**Q4.0: Can you describe what segmentation is in your own words?**

type here

**Q4.1: Devise an experimental set-up in which microscopy and its tools can be used to study symbiosis?**

type here

**Q4.2: Think of a data extraction method. How can we quantify the symbiotic relationship acquired by your new symbiotic set-up?**

type here

***

You have reached the end of this notebook, have you understood the main concepts in this introductions about image analysis?

- What is segmentation?

- How can you extract data from segmentation?

- What is napari python viewer?

- What are plugins and how can I use them to perform image analysis.